Join the data from Part 1 with the data from Part 2 to create a new dataframe.

NOTE:

We constructed our final dataframe for both the Foursquare and YELP data in the process of querying each API - this meant we didnt have two seperate datasets and thus we didnt have to do any joining.

In more detail, we queried the cityBikes dataset filtering on Toronto (network provider Bike Share Toronto) and extracted from that query the latitude and longitude of each bike 
station in the network (717 in total) and the availibility of bikes at each station (we queried in the late afternoon so figures are reflective of that time of day).

With this in hand, we extracted the individual latitude and longitudes from the city bikes dataframe, and concatinated them into a single 'lat, long' value, and using a for-loop, itereated through all of them and
fed them into both the Foresquare and YELP API's, specifiying 1000m (1km) radius, and from each API, extracting the bars and librarys in that radius.

What this allowed us to do:

- Ensure data integrity and quality from the get go - we didnt have to worry about duplicates as the data retrieved from city bikes was entirely unique and thus so was the date subsequently retrieved from the Foursquare and YELP API's. We did it all upstream effectively.
- Constructing our final dataset as a product of the Foursquare and YELP API's, we produced one DataFrame which had all the city bike station cooridnates and amount of free bikes as well as all the data we wanted from both the Foursquare and YELP results. 
This ultimatley negates the need for any pandas merges or SQL joins which could bring about messy joins with differences in keys etc..
- Given that we fed the city bikes cooridnates in the same order into each API, our result DataFrames show the exact same city bike stations in the exact same order which allows for a perfect side by side comparison (we should note that because of YELP's 500 rate limit, we could only query the first 250 bike stations, 1 call for bars and 1 call for libraries).




# Database

Placing our datasets into a SQLite3 database

In [1]:
import pandas as pd
import sqlite3

In [2]:
# Reading in both Foursquare and YELP csv's.

fsq_BarLibs_BikeRadius = pd.read_csv('fsq_BarLibs_BikeRadius.csv')
yelp_BarLibs_BikeRadius1 = pd.read_csv('yelp_BarLibs_BikeRadius1.csv')

In [3]:
fsq_BarLibs_BikeRadius

,Unnamed: 0,bike_station#,latitude,longitude,free_bikes,num_of_bars,min_bar_dist,num_of_libs,min_libs_dist
0,0,0,43.665269,-79.319796,18,11,608.0,2,810.0
1,1,1,43.671420,-79.445947,2,19,292.0,1,664.0
2,2,2,43.666224,-79.317693,10,8,413.0,2,766.0
3,3,3,43.653236,-79.376716,4,50,122.0,9,302.0
4,4,4,43.663722,-79.380288,23,46,43.0,6,555.0
...,...,...,...,...,...,...,...,...,...
712,712,712,43.601915,-79.499419,1,0,NaN,1,885.0
713,713,713,43.642852,-79.531995,1,8,159.0,0,NaN
714,714,714,43.650945,-79.379498,0,50,139.0,0,NaN
715,715,715,43.707356,-79.437231,0,0,NaN,0,NaN


In [4]:
yelp_BarLibs_BikeRadius1

,Unnamed: 0,bike_station#,latitude,longitude,free_bikes,num_of_bars,avg_bar_rating,min_bar_dist,num_of_libs,avg_lig_rating,min_lib_dist
0,0,0,43.665269,-79.319796,18,16,4.00000,216.73,1,3.500000,789.09
1,1,1,43.671420,-79.445947,2,10,3.05000,598.29,2,3.750000,688.66
2,2,2,43.666224,-79.317693,10,16,3.71875,341.48,1,3.500000,765.56
3,3,3,43.653236,-79.376716,4,50,3.87000,130.70,10,3.500000,561.35
4,4,4,43.663722,-79.380288,23,50,3.66000,62.99,9,3.944444,594.79
...,...,...,...,...,...,...,...,...,...,...,...
245,245,245,43.685569,-79.408019,0,2,3.50000,800.23,0,NaN,NaN
246,246,246,43.651678,-79.375233,1,50,3.80000,70.31,8,3.562500,435.89
247,247,247,43.651838,-79.378743,1,50,3.86000,89.87,8,3.812500,399.87
248,248,248,43.665221,-79.394009,1,50,3.73000,357.20,19,3.973684,166.38


In [11]:
# Establing a connection to the SQLite databate

conn = sqlite3.connect('YELP_FSQ_.db')

In [12]:
# inserting both the Foursquare and YELP DataFrames into the created database

fsq_BarLibs_BikeRadius.to_sql('fsq_BarLibs_BikeRadius', conn, if_exists='replace', index=False)
yelp_BarLibs_BikeRadius1.to_sql('yelp_BarLibs_BikeRadius1', conn, if_exists='replace', index=False)


250

In [13]:
# Finally closing the connection.

conn.close()